In [1]:
!pip install transformers

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
!pip install adabelief-pytorch==0.2.0

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [5]:
#!pip install nbformat # 설치 시도 
!pip install -U nbformat # 기설치시, update 

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 77 kB 12.7 MB/s eta 0:00:01
     |████████████████████████████████| 116 kB 33.7 MB/s eta 0:00:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  NOTE: The current PATH contains path(s) starting with `~`, which may not be expanded by all applications.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
nbclient 0.5.1 requires jupyter-client>=6.1.5, but you have jupyter-client 6.0.0 which is incompatible.
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [1]:
import os
import re
import numpy as np
from tqdm import tqdm
from glob import glob
import json
import requests
import tensorflow as tf
from transformers import BertModel, TFBertModel, TFRobertaModel, RobertaTokenizer
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModel, AutoModelForSequenceClassification
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,KFold,StratifiedKFold
from adabelief_pytorch import AdaBelief
from transformers.optimization import get_cosine_schedule_with_warmup
from tqdm import tqdm, tqdm_notebook
import shutil
import gc
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from sklearn.metrics import precision_score, recall_score, f1_score, classification_report

"klue/roberta-large" 사용 시 "klue/roberta-large" 임베딩과 NUM_EPOCHS = 3

"monologg/kobigbird-bert-base" 모델 사용시  "monologg/kobigbird-bert-base" 임베딩으로  NUM_EPOCHS = 6

In [2]:
#random seed 고정
tf.random.set_seed(1234)
np.random.seed(1234)
BATCH_SIZE = 32
NUM_EPOCHS = 3

L_RATE = 1e-5
MAX_LEN = 128
max_grad_norm=1
log_interval=200
NUM_CORES = os.cpu_count()
device = torch.device("cuda:0")

DATA_IN_PATH = './data'
DATA_OUT_PATH = "./data"

In [3]:
#두가지 임베딩
#"klue/roberta-large"
#"monologg/koelectra-base-v3-discriminator"
#tokenizer = AutoTokenizer.from_pretrained("monologg/kobigbird-bert-base", cache_dir='bert_ckpt', do_lower_case=False)
tokenizer = AutoTokenizer.from_pretrained("klue/roberta-large", cache_dir='bert_ckpt', do_lower_case=False)
#tokenizer = AutoTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator", cache_dir='bert_ckpt', do_lower_case=False)

In [4]:
PATH = 'numpy값'

In [5]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')
submission = pd.read_csv('./data/sample_submission.csv')

In [6]:


# 2. Label Encoding (유형, 극성, 시제, 확실성)
from sklearn.preprocessing import LabelEncoder
type_le = LabelEncoder()
train["유형"] = type_le.fit_transform(train["유형"].values)


polarity_le = LabelEncoder()
train["극성"] = polarity_le.fit_transform(train["극성"].values)


tense_le = LabelEncoder()
train["시제"] = tense_le.fit_transform(train["시제"].values)


certainty_le = LabelEncoder()
train["확실성"] = certainty_le.fit_transform(train["확실성"].values)


In [7]:
class TrainDataset(Dataset):
    def __init__(self, df):
        self.df_data = df
    def __getitem__(self, index):
        # get the sentence from the dataframe
        sentence = self.df_data.loc[index, '문장']
        encoded_dict = tokenizer(
          text = sentence,
          add_special_tokens = True, 
          max_length = MAX_LEN,
          pad_to_max_length = True,
          truncation=True,           # Pad & truncate all sentences.
          return_tensors="pt")

        padded_token_list = encoded_dict['input_ids'][0]
        token_type_id = encoded_dict['token_type_ids'][0]
        att_mask = encoded_dict['attention_mask'][0]
        target = torch.tensor(self.df_data.loc[index, "확실성"])
        sample = (padded_token_list, token_type_id , att_mask, target)
        return sample
    def __len__(self):
        return len(self.df_data)

In [8]:
class TestDataset(Dataset):
    def __init__(self, df):
        self.df_data = df
    def __getitem__(self, index):
        # get the sentence from the dataframe
        sentence = self.df_data.loc[index, '문장']
        encoded_dict = tokenizer(
          text = sentence,
          add_special_tokens = True, 
          max_length = MAX_LEN,
          pad_to_max_length = True,
          truncation=True,           # Pad & truncate all sentences.
          return_tensors="pt")

        padded_token_list = encoded_dict['input_ids'][0]
        token_type_id = encoded_dict['token_type_ids'][0]
        att_mask = encoded_dict['attention_mask'][0]
        sample = (padded_token_list, token_type_id , att_mask)
        return sample
    def __len__(self):
        return len(self.df_data)

In [9]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

h값으로 미세조정한 후 저장
ex) Roberta229 = h값 229

In [17]:
#332 : epoch 3 train acc 0.9187989758050054
# 273 : epoch 3 train acc 0.9268683707635386
# 325 : epoch 3 train acc 0.9205169095085536
# 251 : epoch 3 train acc 0.9313042428174262
# kobig120 : epoch 3 train acc 0.9093886905876433

#극성
#332 : epoch 3 train acc 0.9854487346754297
#273 : epoch 3 train acc 0.9868469425354284
#325 : epoch 3 train acc 0.984686812931019
#251 : epoch 3 train acc 0.988702150120287
# kobig120 : epoch 3 train acc 0.9812677962160321
#시제
#332 : epoch 3 train acc 0.919275517196581
#273 : epoch 3 train acc 0.9312424963644657
#325 : epoch 3 train acc 0.9180134893123439
#251 : epoch 3 train acc 0.9349046320802006
#

#확실성
# 332 : epoch 3 train acc 0.9496036063037623
# 273 : epoch 3 train acc 0.9542198576228881
# 325 : epoch 3 train acc 0.9492931109324948
# 251 : epoch 3 train acc 0.9584442313345445
#AUto 
train_data = TrainDataset(train)

test_data = TestDataset(test)

train_dataloader = torch.utils.data.DataLoader(train_data,
                                        batch_size=BATCH_SIZE,
                                        shuffle=True,
                                      num_workers=NUM_CORES)
test_dataloader = torch.utils.data.DataLoader(test_data,
                                        batch_size=BATCH_SIZE,
                                        shuffle=False,
                                      num_workers=NUM_CORES)
#두가지 모델
#"klue/roberta-large"
#"monologg/koelectra-base-v3-discriminator"
#kobigbird-bert-base
#model = AutoModelForSequenceClassification.from_pretrained("skt/kogpt2-base-v2",um_labels=3)
model = AutoModelForSequenceClassification.from_pretrained("klue/roberta-large",num_labels=2)


####미세조정
n=0
for name, child in model.named_children():
    if n==0:
      h=0
      for param in child.parameters():
        if h<=273: #이부분 숫자 조절로 fine-tuning => Roberta229: h=229
          param.requires_grad = False
        h+=1
    n+=1
#####
    # print(param)
model.to(device)
optimizer = AdaBelief(model.parameters(), lr=1e-5, eps=1e-16, betas=(0.9,0.999), weight_decouple = True, rectify = False)

warmup_ratio = 0.1
t_total = len(train_dataloader) * NUM_EPOCHS
warmup_step = int(t_total * warmup_ratio)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

for e in range(NUM_EPOCHS):
    train_acc = 0.0
    test_acc = 0.0
    best_acc =0.0
    model.train()
    torch.set_grad_enabled(True)
    for batch_id, (input_id,token_type_id,attention_mask,label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        input_id = input_id.long().to(device)
        token_type_id = token_type_id.long().to(device)
        attention_mask = attention_mask.long().to(device)
        label = label.to(device)
        outputs = model(input_ids=input_id, token_type_ids=token_type_id, attention_mask=attention_mask, labels=label)
        loss = outputs[0]
        out = outputs[1]
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        _, preds = torch.max(out, dim=1)
        train_acc += f1_score(preds.cpu(), label.cpu(),average='weighted')
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

preds = [] 
model.eval()
torch.set_grad_enabled(False)
for batch_id, (input_id,token_type_id,attention_mask) in enumerate(tqdm_notebook(test_dataloader)):
    input_id = input_id.long().to(device)
    token_type_id = token_type_id.long().to(device)
    attention_mask = attention_mask.long().to(device)
    outputs = model(input_ids=input_id, token_type_ids=token_type_id, attention_mask=attention_mask)
    out = outputs[0]
    for inp in out:
      preds.append(inp.detach().cpu().numpy())
Preds = np.array(preds)

/home/work/.local/lib/python3.8/site-packages/transformers/configuration_utils.py:336: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceC

Please check your arguments if you have upgraded adabelief-pytorch from version 0.0.5.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  ---------
adabelief-pytorch=0.0.5  1e-08  False              False
>=0.1.0 (Current 0.2.0)  1e-16  True               True
SGD better than Adam (e.g. CNN for Image Classification)    Adam better than SGD (e.g. Transformer, GAN)
----------------------------------------------------------  ----------------------------------------------
Recommended eps = 1e-8                                      Recommended eps = 1e-16
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer
You can disable the log message by setting "print_change_log = False", though it is recommended to keep as a reminder.

Weight decoupling enabled in AdaBelief


  0%|          | 0/517 [00:00<?, ?it/s]

/home/work/.local/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2211: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/home/work/.local/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2211: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the ma

epoch 1 batch id 1 loss 0.7336499691009521 train acc 0.193525641025641
epoch 1 batch id 201 loss 0.36710068583488464 train acc 0.9052488664750682
epoch 1 batch id 401 loss 0.06344197690486908 train acc 0.923247490981822
epoch 1 train acc 0.9280391852041546


...

/home/work/.local/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2211: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/home/work/.local/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2211: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the ma

epoch 3 batch id 1 loss 0.28200000524520874 train acc 0.9041666666666667
epoch 3 batch id 201 loss 0.04326974228024483 train acc 0.9538954649048361
epoch 3 batch id 401 loss 0.16976340115070343 train acc 0.9542177140868108
epoch 3 train acc 0.9542198576228881


<ipython-input-17-37a64307fcf9>:94: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (input_id,token_type_id,attention_mask) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/222 [00:00<?, ?it/s]

/home/work/.local/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2211: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/home/work/.local/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2211: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the ma

In [40]:
results = np.argmax(Preds, axis=1)
#submission['topic_idx']= results

In [98]:
#test_pred=type_le.inverse_transform(results)
polarity_pred=polarity_le.inverse_transform(results)
#tense_pred=tense_le.inverse_transform(results)
#certainty_le_pred=certainty_le.inverse_transform(results)

In [47]:
submission['유형']= test_pred

In [99]:
polarity_pred

array(['긍정', '긍정', '긍정', ..., '긍정', '긍정', '긍정'], dtype=object)

In [29]:
submission.유형.value_counts()

사실형    7090
Name: 유형, dtype: int64

In [100]:
submission['극성']= polarity_pred

In [101]:
submission.극성.value_counts()

긍정    6754
부정     287
미정      49
Name: 극성, dtype: int64

In [78]:
submission['시제']= tense_pred

In [79]:
submission.시제.value_counts()

과거    3462
현재    2961
미래     667
Name: 시제, dtype: int64

In [81]:
submission['확실성']= certainty_le_pred

In [84]:
submission.확실성.value_counts()

확실     6627
불확실     463
Name: 확실성, dtype: int64

In [87]:
submission['label']=submission.유형+'-'+submission.극성+'-'+submission.시제+'-'+submission.확실성

In [89]:
last=submission[['ID','label']]
last=last.set_index('ID')
last.to_csv('klue_최종label.csv',encoding='utf-8-sig',)

In [11]:
Roberta332_certainty=Preds 
np.save(PATH+'Roberta332_certainty.npy',arr=Roberta332_certainty)

In [18]:
Roberta273_certainty=Preds 
np.save(PATH+'Roberta273_certainty.npy',arr=Roberta273_certainty)

In [38]:
Roberta325_tense=Preds 
np.save(PATH+'Roberta325_tense.npy',arr=Roberta325_tense)

In [14]:
Roberta251_certainty=Preds
np.save(PATH+'Roberta251_certainty.npy',arr=Roberta251_certainty)

In [69]:
kobig120=Preds 
np.save(PATH+'Electra120.npy',arr=kobig120)

In [ ]:
#332 : epoch 3 train acc 0.9187989758050054
# 273 : epoch 3 train acc 0.9268683707635386
# 325 : epoch 3 train acc 0.9205169095085536
# 251 : epoch 3 train acc 0.9313042428174262
# kobig120 : epoch 3 train acc 0.9093886905876433

#극성
#332 : epoch 3 train acc 0.9854487346754297
#273 : epoch 3 train acc 0.9868469425354284
#325 : epoch 3 train acc 0.984686812931019
#251 : epoch 3 train acc 0.988702150120287
# kobig120 : epoch 3 train acc 0.9812677962160321
#시제
#332 : epoch 3 train acc 0.919275517196581
#273 : epoch 3 train acc 0.9312424963644657
#325 : epoch 3 train acc 0.9180134893123439
#251 : epoch 3 train acc 0.9349046320802006
#

#확실성
# 332 : epoch 3 train acc 0.9496036063037623
# 273 : epoch 3 train acc 0.9539584020942307
# 325 : epoch 3 train acc 0.9492931109324948
# 251 : epoch 3 train acc 0.9584442313345445

In [19]:
kobig120=np.load('numpy값Electra120.npy')
Roberta251=np.load('numpy값Roberta251.npy')
Roberta273=np.load('numpy값Roberta273.npy')
Roberta325=np.load('numpy값Roberta325.npy')
Roberta332=np.load('numpy값Roberta332_type.npy')

Roberta332_polarity= np.load('numpy값Roberta332_polarity.npy')
Roberta325_polarity= np.load('numpy값Roberta325_polarity.npy')
Roberta273_polarity= np.load('numpy값Roberta273_polarity.npy')
Roberta251_polarity= np.load('numpy값Roberta251_polarirty.npy')
kobig120_polarity=np.load('numpy값kobig120_polarity.npy')

Roberta332_tense= np.load('numpy값Roberta332_tense.npy')
Roberta325_tense= np.load('numpy값Roberta325_tense.npy')
Roberta273_tense= np.load('numpy값Roberta273_tense.npy')
Roberta251_tense= np.load('numpy값Roberta251_tense.npy')
kobig120_tense=np.load('numpy값kobig120_tense.npy')

Roberta332_certainty= np.load('numpy값Roberta332_certainty.npy')
Roberta325_certainty= np.load('numpy값Roberta325_certainty.npy')
Roberta273_certainty= np.load('numpy값Roberta273_certainty.npy')
Roberta251_certainty= np.load('numpy값Roberta251_certainty.npy')
kobig120_certainty=np.load('numpy값kobig120_certainty.npy')


In [20]:
Pred_values_type = Roberta332*0.1 + Roberta325 * 0.15 + Roberta273 * 0.15 + Roberta251 * 0.4 + kobig120 * 0.2
Pred_values_polarity = Roberta332_polarity*0.1 + Roberta325_polarity * 0.15 + Roberta273_polarity * 0.15 + Roberta251_polarity * 0.4 + kobig120_polarity * 0.2
Pred_values_tense = Roberta332_tense*0.1 + Roberta325_tense * 0.15 + Roberta273_tense * 0.15 + Roberta251_tense * 0.4 + kobig120_tense * 0.2
Pred_values_certainty = Roberta332_certainty*0.1 + Roberta325_certainty * 0.15 + Roberta273_certainty * 0.15 + Roberta251_certainty * 0.4 + kobig120_certainty * 0.2

In [24]:
results_type = np.argmax(Pred_values_type, axis=1)
type_pred_ang=type_le.inverse_transform(results_type)
submission['유형']= type_pred_ang

results_polarity = np.argmax(Pred_values_polarity, axis=1)
polarity_pred_ang=polarity_le.inverse_transform(results_polarity)
submission['극성']= polarity_pred_ang

results_tense = np.argmax(Pred_values_tense, axis=1)
tense_pred_ang=tense_le.inverse_transform(results_tense)
submission['시제']= tense_pred_ang

results_certainty = np.argmax(Pred_values_certainty, axis=1)
certainty_pred_ang=certainty_le.inverse_transform(results_certainty)
submission['확실성']= certainty_pred_ang

submission['label']=submission.유형+'-'+submission.극성+'-'+submission.시제+'-'+submission.확실성
last=submission[['ID','label']]
last=last.set_index('ID')
last.to_csv('klue_kobig_ang_최종label.csv',encoding='utf-8-sig',)
#submission.to_csv(PATH + 'daconnews.csv', index=False)

In [25]:
submission

,ID,label,유형,극성,시제,확실성
0,TEST_0000,사실형-긍정-현재-확실,사실형,긍정,현재,확실
1,TEST_0001,사실형-긍정-현재-확실,사실형,긍정,현재,확실
2,TEST_0002,사실형-긍정-과거-확실,사실형,긍정,과거,확실
3,TEST_0003,사실형-긍정-현재-확실,사실형,긍정,현재,확실
4,TEST_0004,사실형-긍정-과거-확실,사실형,긍정,과거,확실
...,...,...,...,...,...,...
7085,TEST_7085,사실형-긍정-현재-확실,사실형,긍정,현재,확실
7086,TEST_7086,추론형-긍정-현재-확실,추론형,긍정,현재,확실
7087,TEST_7087,사실형-긍정-현재-확실,사실형,긍정,현재,확실
7088,TEST_7088,추론형-긍정-미래-확실,추론형,긍정,미래,확실
